# Local feature extractor (LFE)

In [27]:
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

learning_rate = 0.00001
batch_size = 8
num_epoch = 1

class LFE(nn.Module):
    def __init__(self):
        super(LFE, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), stride=(1, 1), dilation = 1,
                               padding='same')
        self.pool_1 = nn.MaxPool2d(2, stride=(2))
        self.conv_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=(1, 1), dilation = 1,
                               padding='same')
        self.pool_2 = nn.MaxPool2d(2, stride=(2))
        self.conv_3 = nn.Conv2d(in_channels=32, out_channels=48, kernel_size=(3, 3), stride=(1, 1), dilation = 1,
                               padding='same')
        self.conv_4 = nn.Conv2d(in_channels=48, out_channels=48, kernel_size=(3, 3), stride=(1, 1), dilation = 1,
                               padding='same')
        self.pool_3 = nn.MaxPool2d(2, stride=(2))
        self.conv_5 = nn.Conv2d(in_channels=48, out_channels=64, kernel_size=(3, 3), stride=(1, 1), dilation = 1,
                               padding='same')
        self.conv_6 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=(1, 1), dilation = 1,
                               padding='same')
        self.conv_7 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=(1, 1), dilation = 1,
                               padding='same')
    def forward(self, x):
        x = self.conv_1(x)
        print("conv_1 ",x.shape)
        x = self.pool_1(x)
        print("pool_1 ",x.shape)
        x = self.conv_2(x)
        print("conv_2 ",x.shape)
        x = self.pool_2(x)
        print("pool_2",x.shape)
        x = self.conv_3(x)
        print("conv_3 ",x.shape)
        x = self.conv_4(x)
        print("conv_4 ",x.shape)
        x = self.pool_3(x)
        print("pool_3 ",x.shape)
        x = self.conv_5(x)
        print("conv_5",x.shape)
        x = self.conv_6(x)
        print("conv_6",x.shape)
        x = self.conv_7(x)
        print("conv_7",x.shape)
        return x


In [28]:
model = LFE()
lfe_out = torch.rand(3,3,768,1024)
lfe_out = model(lfe_out)

conv_1  torch.Size([3, 16, 768, 1024])
pool_1  torch.Size([3, 16, 384, 512])
conv_2  torch.Size([3, 32, 384, 512])
pool_2 torch.Size([3, 32, 192, 256])
conv_3  torch.Size([3, 48, 192, 256])
conv_4  torch.Size([3, 48, 192, 256])
pool_3  torch.Size([3, 48, 96, 128])
conv_5 torch.Size([3, 64, 96, 128])
conv_6 torch.Size([3, 64, 96, 128])
conv_7 torch.Size([3, 64, 96, 128])


# Hybrid dilation convolution (HDC) block

In [29]:
class HDC(nn.Module):
    def __init__(self):
        super(HDC, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=64, out_channels=16, kernel_size=(1, 1), stride=(1, 1), dilation = 1,
                               padding='same')
        self.conv_2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), stride=(1, 1), dilation = 1,
                               padding='same')
        self.conv_3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), stride=(1, 1), dilation = 2,
                               padding='same')
        self.conv_4 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), stride=(1, 1), dilation = 3,
                               padding='same')
        
    def forward(self,i):
        print("input shape i ",i.shape)
        i_0 = self.conv_1(i)
        print("conv_1 ",i_0.shape)
        i_1 = self.conv_2(i_0)
        print("conv_2 ",i_1.shape)
        i_2 = self.conv_3(i_1)
        print("conv_3 ",i_2.shape)
        i_3 = self.conv_4(i_2)
        print("conv_4 ",i_3.shape)
        c = torch.cat((i_0,i_1,i_2,i_3),dim=1)
        out = c+i
        print("after skip connection addition",out.shape)
        
        return out


In [30]:
model = HDC()
hdc_out = model(lfe_out)


input shape i  torch.Size([3, 64, 96, 128])
conv_1  torch.Size([3, 16, 96, 128])
conv_2  torch.Size([3, 16, 96, 128])
conv_3  torch.Size([3, 16, 96, 128])
conv_4  torch.Size([3, 16, 96, 128])
after skip connection addition torch.Size([3, 64, 96, 128])


# Efficient attention-based fusion (EAF) block

In [40]:
class EAF(nn.Module):
    def __init__(self):
        super(EAF, self).__init__()
        
        self.glb_avg_pool_w1 = nn.AvgPool2d((96,128))
        self.glb_avg_pool_w2 = nn.AvgPool2d((96,128))

        self.conv_w1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=(9), stride=(1),
                               padding='same')
        self.conv_w2 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=(9), stride=(1),
                               padding='same')
        
        self.norm = nn.InstanceNorm2d(64)
        
        self.conv_out = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(1), stride=(1, 1),
                               padding='same')

    def forward(self,x1,x2):
        print("x1_shape ",x1.shape)
        print("x2_shape ",x2.shape)

        w1 = self.glb_avg_pool_w1(x1)
        print("after global average pool w1 ",w1.shape)
        w2 = self.glb_avg_pool_w1(x2)
        print("after global average pool w1 ",w2.shape)
        
        w1 = torch.reshape(w1,(w1.shape[0],w1.shape[1],1))
        print("reshape w1 ",w1.shape)
        w2 = torch.reshape(w2,(w2.shape[0],w2.shape[1],1))
        print("reshape w2 ",w2.shape)
        w = torch.cat((w1,w2),dim=1)
        print("concated w1 and w2 ",w.shape)
        
        w = w.reshape(w.shape[0],w.shape[2],w.shape[1])
        
        w1 = self.conv_w1(w)
        print("conv_1 ",w1.shape)
        w2 = self.conv_w2(w)
        print("conv_1 ",w2.shape)
        
        w1 = F.softmax(torch.reshape(w1,(w1.shape[0],w1.shape[3],1,1)),dim=1) 
        print("conv_1 ",w1.shape)
        w2 = F.softmax(torch.reshape(w2,(w2.shape[0],w2.shape[3],1,1)),dim=1)
        print("conv_1 ",w2.shape)
        
        w1 = torch.mul(w1, x1)
        print("conv_1 ",w1.shape)
        w2 = torch.mul(w2, x2)
        print("conv_1 ",w2.shape)
        
        out = torch.sum(w1, w2, x1, x2)
        print("conv_1 ",i_0.shape)
        
        out = self.norm(out)
        print("conv_1 ",i_0.shape)
        
        out = self.conv_out(out)
        print("conv_1 ",i_0.shape)
        
        return out

In [41]:
model = EAF()
print(model(hdc_out,lfe_out))

x1_shape  torch.Size([3, 64, 96, 128])
x2_shape  torch.Size([3, 64, 96, 128])
after global average pool w1  torch.Size([3, 64, 1, 1])
after global average pool w1  torch.Size([3, 64, 1, 1])
reshape w1  torch.Size([3, 64, 1])
reshape w2  torch.Size([3, 64, 1])
concated w1 and w2  torch.Size([3, 128, 1])
conv_1  torch.Size([3, 1, 128])
conv_1  torch.Size([3, 1, 128])


IndexError: tuple index out of range

In [ ]:
m = nn.InstanceNorm2d(100)
input = torch.randn(20, 100, 35, 45)
output = m(input)
print(output.shape)

# Scale feature fusion module (SFFM)

In [ ]:
class SFFM(nn.Module):
    def __init__(self,lfe,hdc2,hdc4,hdc6):
        super(SFFM, self).__init__()
        self.lfe = lfe
        self.hdc2 = hdc2
        self.hdc4 = hdc4
        self.hdc6 = hdc6
        
        self.eaf1 = EAF()
        self.eaf2 = EAF()
        self.eaf3 = EAF()


    def forward(self):
        lfe = self.lfe
        print("lfe shape ",lfe.shape)
        hdc2 = self.hdc2 
        print("hdc2 shape ",hdc2.shape)
        hdc4 = self.hdc4
        print("hdc4 shape ",hdc4.shape)
        hdc6 = self.hdc6  
        print("hdc6 shape ",hdc6.shape)
        
        feature_map = self.eaf1(lfe,hdc2)
        print("feature_map shape",feature_map.shape)
        feature_map = self.eaf2(feature_map,hdc4)
        print("feature_map shape",feature_map.shape)
        feature_map = self.eaf3(feature_map,hdc6)
        print("feature_map shape",feature_map.shape)
        
        out = feature_map + lfe
        print("out shape ",out.shape)
        
        return out

In [ ]:
model = SFFM(lfe_out,hdc_out,hdc_out,hdc_out)
model = model()

# Density map regressor (DMR)

In [42]:
class DMR(nn.Module):
    def __init__(self):
        super(DMR, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(3, 3), stride=(1, 1), dilation = 1,
                               padding='same')
        self.deconv_1 = nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=(2, 2), stride=(2,2))
        self.conv_2 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(3, 3), stride=(1, 1), dilation = 1,
                               padding='same')
        self.deconv_2 = nn.ConvTranspose2d(in_channels=8, out_channels=4, kernel_size=(2, 2), stride=(2,2))
        self.conv_3 = nn.Conv2d(in_channels=4, out_channels=4, kernel_size=(3, 3), stride=(1, 1), dilation = 2,
                               padding='same')
        self.deconv_3 = nn.ConvTranspose2d(in_channels=4, out_channels=1, kernel_size=(2, 2), stride=(2,2))
        
    def forward(self,x):
        print("input shape x ",x.shape)
        x = self.conv_1(x)
        print("conv_1 ",x.shape)
        x = self.deconv_1(x)
        print("deconv_1 ",x.shape)
        x = self.conv_2(x)
        print("conv_2 ",x.shape)
        x = self.deconv_2(x)
        print("deconv_2 ",x.shape)
        x = self.conv_3(x)
        print("conv_3 ",x.shape)
        x = self.deconv_3(x)
        print("deconv_3 ",x.shape)
        
        return x


In [43]:
model = DMR()
lfe_out = torch.rand(3,64,96,128)
lfe_out = model(lfe_out)

input shape x  torch.Size([3, 64, 96, 128])
conv_1  torch.Size([3, 32, 96, 128])
deconv_1  torch.Size([3, 16, 192, 256])
conv_2  torch.Size([3, 8, 192, 256])
deconv_2  torch.Size([3, 4, 384, 512])
conv_3  torch.Size([3, 4, 384, 512])
deconv_3  torch.Size([3, 1, 768, 1024])


In [ ]:
def isPalindrome(s):
    return s == s[::-1]

def spiralPrint(a):
    k = 0
    l = 0
    m = len(a)
    n = len(a[0])
    ans = []
    st = ''
 
    while (k < m and l < n):
 
        for i in range(l, n):
            st += a[k][i]
            if len(st)>1 and isPalindrome(st):
                ans.append(st)
                st=''
            
        k += 1
 
        for i in range(k, m):
            st += a[i][n - 1]
            if len(st)>1 and isPalindrome(st):
                ans.append(st)
                st=''
 
        n -= 1
 
        if (k < m):
 
            for i in range(n - 1, (l - 1), -1):
                st += a[m - 1][i]
                if len(st)>1 and isPalindrome(st):
                    ans.append(st)
                    st=''
 
            m -= 1
 
        if (l < n):
            for i in range(m - 1, k - 1, -1):
                st += a[i][l]
                if len(st)>1 and isPalindrome(st):
                    ans.append(st)
                    st=''
 
            l += 1
    
    
    ans.append(st)        
    return ans




input_arr = []
m = int(input("number of rows"))
for i in range(m):
    st = input("enter string ")
    input_arr.append(st.split(','))

print(input_arr)
outstr = spiralPrint(input_arr)

ans = outstr[0]

for i in range(len(outstr)-1):
    ans+=','
    ans+=outstr[i+1]
    
print(ans)
 

In [ ]:
def power(x, y):
      
    if y == 0:
        return 1
    if y % 2 == 0:
        return power(x, y // 2) * power(x, y // 2)
          
    return x * power(x, y // 2) * power(x, y // 2)
  
# Function to calculate order of the number
def order(x):
  
    # Variable to store of the number
    n = 0
    while (x != 0):
        n = n + 1
        x = x // 10
          
    return n
  
# Function to check whether the given 
# number is Armstrong number or not
def isArmstrong(x):
      
    n = order(x)
    temp = x
    sum1 = 0
      
    while (temp != 0):
        r = temp % 10
        sum1 = sum1 + power(r, n)
        temp = temp // 10
  
    # If condition satisfies
    return (sum1 == x)

instr = input("input string")
ascii_val = [ord(ch) for ch in instr]
total_sum = sum(ascii_val)


if isArmstrong(total_sum):
    check={}
    outstr=''
    for ele in ascii_val:
        if chr(ele) in check:
            pass
        else:
            outstr+=chr(ele)
            check[chr(ele)]=1
    print(outstr)
else:
    print('-1')
